# Laborator 6: Evaluare Comparativă și Metrici de Performanță

## Obiective
- Compararea tuturor modelelor antrenate (ML clasic + Deep Learning)
- Înțelegerea metricilor de performanță
- Vizualizarea rezultatelor
- Interpretarea și concluzii

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                            confusion_matrix, classification_report, roc_curve, auc,
                            precision_recall_curve)

plt.style.use('seaborn-v0_8-whitegrid')
print("Biblioteci încărcate!")

## 1. Încărcarea Datelor și Modelelor

In [ ]:
# Încărcare date test
try:
    X_test = np.load('X_test.npy')
    y_test = np.load('y_test.npy')
except:
    from sklearn.datasets import make_classification
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    X, y = make_classification(n_samples=10000, n_features=38, n_classes=2, random_state=42)
    X = StandardScaler().fit_transform(X)
    _, X_test, _, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Test set: {X_test.shape}")

In [ ]:
# Încărcare modele
models = {}
predictions = {}

# ML Clasic
try:
    with open('decision_tree_model.pkl', 'rb') as f:
        models['Decision Tree'] = pickle.load(f)
    with open('random_forest_model.pkl', 'rb') as f:
        models['Random Forest'] = pickle.load(f)
    with open('knn_model.pkl', 'rb') as f:
        models['KNN'] = pickle.load(f)
except:
    print("Modelele ML clasic nu au fost găsite. Creăm modele noi...")
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.model_selection import train_test_split
    
    X_train, _, y_train, _ = train_test_split(
        np.vstack([X_test, X_test]), np.hstack([y_test, y_test]), test_size=0.5, random_state=42
    )
    
    models['Decision Tree'] = DecisionTreeClassifier(max_depth=10, random_state=42).fit(X_train, y_train)
    models['Random Forest'] = RandomForestClassifier(n_estimators=100, random_state=42).fit(X_train, y_train)
    models['KNN'] = KNeighborsClassifier(n_neighbors=5).fit(X_train, y_train)

# Deep Learning
try:
    import tensorflow as tf
    models['MLP'] = tf.keras.models.load_model('mlp_model.keras')
    models['LSTM'] = tf.keras.models.load_model('lstm_model.keras')
except:
    print("Modelele Deep Learning nu au fost găsite.")

print(f"\nModele încărcate: {list(models.keys())}")

In [ ]:
# Generare predicții
for name, model in models.items():
    if 'MLP' in name:
        pred = (model.predict(X_test) > 0.5).astype(int).flatten()
    elif 'LSTM' in name:
        X_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
        pred = (model.predict(X_lstm) > 0.5).astype(int).flatten()
    else:
        pred = model.predict(X_test)
    predictions[name] = pred
    print(f"{name}: {len(pred)} predicții")

## 2. Matricea de Confuzie

In [ ]:
# Afișăm matricele de confuzie pentru toate modelele
n_models = len(predictions)
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, (name, y_pred) in enumerate(predictions.items()):
    if i >= 6:
        break
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                xticklabels=['Normal', 'Attack'],
                yticklabels=['Normal', 'Attack'])
    axes[i].set_title(f'{name}')
    axes[i].set_ylabel('Actual')
    axes[i].set_xlabel('Predicted')

# Ascundem axele neutilizate
for j in range(i+1, 6):
    axes[j].axis('off')

plt.tight_layout()
plt.suptitle('Matricele de Confuzie - Toate Modelele', y=1.02, fontsize=14)
plt.show()

## 3. Calcularea și Compararea Metricilor

In [ ]:
# Calculăm toate metricile
results = []

for name, y_pred in predictions.items():
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    results.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'True Positives': tp,
        'True Negatives': tn,
        'False Positives': fp,
        'False Negatives': fn
    })

results_df = pd.DataFrame(results)
print("\n" + "="*80)
print("REZULTATE COMPARATIVE")
print("="*80)
print(results_df.to_string(index=False))

In [ ]:
# Vizualizare comparativă
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Grafic 1: Toate metricile
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
x = np.arange(len(results_df))
width = 0.2
colors = ['steelblue', 'forestgreen', 'coral', 'purple']

for i, metric in enumerate(metrics):
    axes[0].bar(x + i*width, results_df[metric], width, label=metric, color=colors[i])

axes[0].set_xlabel('Model')
axes[0].set_ylabel('Scor')
axes[0].set_title('Comparație Metrici - Toate Modelele')
axes[0].set_xticks(x + width * 1.5)
axes[0].set_xticklabels(results_df['Model'], rotation=45, ha='right')
axes[0].legend()
axes[0].set_ylim([0.7, 1.0])

# Grafic 2: F1-Score ranking
sorted_df = results_df.sort_values('F1-Score', ascending=True)
colors_bar = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(sorted_df)))
axes[1].barh(sorted_df['Model'], sorted_df['F1-Score'], color=colors_bar)
axes[1].set_xlabel('F1-Score')
axes[1].set_title('Ranking Modele după F1-Score')
axes[1].set_xlim([0.7, 1.0])

for i, v in enumerate(sorted_df['F1-Score']):
    axes[1].text(v + 0.01, i, f'{v:.4f}', va='center')

plt.tight_layout()
plt.show()

## 4. Curba ROC și AUC

In [ ]:
# Curba ROC pentru modelele care suportă predict_proba
plt.figure(figsize=(10, 8))

for name, model in models.items():
    try:
        if 'MLP' in name:
            y_proba = model.predict(X_test).flatten()
        elif 'LSTM' in name:
            X_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
            y_proba = model.predict(X_lstm).flatten()
        elif hasattr(model, 'predict_proba'):
            y_proba = model.predict_proba(X_test)[:, 1]
        else:
            continue
        
        fpr, tpr, _ = roc_curve(y_test, y_proba)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, linewidth=2, label=f'{name} (AUC = {roc_auc:.4f})')
    except Exception as e:
        print(f"Nu s-a putut calcula ROC pentru {name}: {e}")

plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random (AUC = 0.5)')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curba ROC - Comparație Modele')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

## 5. Analiza FP vs FN (Important pentru Securitate!)

In [ ]:
# În securitate: FN (atacuri nedetectate) sunt mai grave decât FP (alarme false)
print("\nAnaliza False Positives vs False Negatives:")
print("="*60)
print("\nÎn detecția intruziunilor:")
print("  - FP (False Positive) = Alarma falsă (trafic normal clasificat ca atac)")
print("  - FN (False Negative) = Atac nedetectat (PERICULOS!)")
print("\n" + "="*60)

fp_fn_df = results_df[['Model', 'False Positives', 'False Negatives', 'Recall']].copy()
fp_fn_df = fp_fn_df.sort_values('False Negatives')
print(fp_fn_df.to_string(index=False))

print("\n" + "="*60)
best_model = fp_fn_df.iloc[0]['Model']
print(f"\nModelul cu cele mai puține atacuri nedetectate: {best_model}")
print(f"(Recall mare = mai puține FN)")

In [ ]:
# Vizualizare FP vs FN
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(results_df))
width = 0.35

bars1 = ax.bar(x - width/2, results_df['False Positives'], width, label='False Positives (Alarme false)', color='orange')
bars2 = ax.bar(x + width/2, results_df['False Negatives'], width, label='False Negatives (Atacuri nedetectate)', color='red')

ax.set_xlabel('Model')
ax.set_ylabel('Număr de erori')
ax.set_title('False Positives vs False Negatives')
ax.set_xticks(x)
ax.set_xticklabels(results_df['Model'], rotation=45, ha='right')
ax.legend()

plt.tight_layout()
plt.show()

## 6. Concluzii și Recomandări

In [ ]:
# Concluzii automate
print("\n" + "="*60)
print("CONCLUZII")
print("="*60)

best_accuracy = results_df.loc[results_df['Accuracy'].idxmax()]
best_f1 = results_df.loc[results_df['F1-Score'].idxmax()]
best_recall = results_df.loc[results_df['Recall'].idxmax()]
best_precision = results_df.loc[results_df['Precision'].idxmax()]

print(f"\n1. Cel mai bun Accuracy: {best_accuracy['Model']} ({best_accuracy['Accuracy']:.4f})")
print(f"2. Cel mai bun F1-Score: {best_f1['Model']} ({best_f1['F1-Score']:.4f})")
print(f"3. Cel mai bun Recall: {best_recall['Model']} ({best_recall['Recall']:.4f})")
print(f"4. Cel mai bun Precision: {best_precision['Model']} ({best_precision['Precision']:.4f})")

print("\nRECOMANDĂRI:")
print(f"- Pentru detectarea atacurilor (minimizare FN): {best_recall['Model']}")
print(f"- Pentru echilibru general: {best_f1['Model']}")
print(f"- Pentru minimizarea alarmelor false: {best_precision['Model']}")

In [ ]:
# Salvare rezultate
results_df.to_csv('final_comparison_results.csv', index=False)
print("\nRezultate salvate în: final_comparison_results.csv")

## Rezumat Final

În acest laborator am:
1. Comparat toate modelele antrenate în laboratoarele anterioare
2. Calculat și interpretat metricile de performanță
3. Vizualizat rezultatele prin grafice și curbe ROC
4. Analizat trade-off-ul FP vs FN în contextul securității

### Metrici importante pentru IDS:
- **Recall** (Sensitivity) - Cât de multe atacuri detectăm
- **Precision** - Cât de siguri suntem când spunem "atac"
- **F1-Score** - Echilibru între Precision și Recall